# Loeys-Dietz syndrome
Loeys-Dietz syndrome (LDS) is an autosomal dominant aortic aneurysm syndrome characterized by multisystemic involvement. The most typical clinical triad includes hypertelorism, bifid uvula or cleft palate and aortic aneurysm with tortuosity. Affected individuals may expereince aortic dissection at smaller aortic diameter and arterial aneurysms throughout the arterial tree. The genetic cause is heterogeneous and includes mutations in genes encoding for components of the transforming growth factor beta (TGFβ) signalling pathway: TGFBR1, TGFBR2, SMAD2, SMAD3, TGFB2 and TGFB3 (See [Velchev JD, et al. (2021). Loeys-Dietz Syndrome. Adv Exp Med Biol](https://pubmed.ncbi.nlm.nih.gov/34807423/)).

This notebook will explore whether there are significant differences in phenotypic features between different genetic forms of LDS. We will include TGFBR1 and TGFBR2 (which were the first forms of LDS to be identified) with 

In [1]:
import gpsea
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()
print(f'Loaded HPO v{hpo.version}')
print(f"Using genophenocorr version {gpsea.__version__}")

Loaded HPO v2025-01-16
Using genophenocorr version 0.9.6.dev0


## LDS1

Loeys Dietz syndrome 1 (LDS1) is caused by mutation in the TGFBR1 gene.

In [2]:
from ppktstore.registry import configure_phenopacket_registry

tgfbr1_symbol = 'TGFBR1'
tgfbr1_mane_tx_id = 'NM_004612.4'
tgfbr1_mane_protein_id = 'NP_004603.1' # TGF-beta receptor type-1 isoform 1 precursor"
lds1_disease_id = "OMIM:609192"


phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store("0.1.24") as ps:
    lds1_phenopackets = tuple(ps.iter_cohort_phenopackets(tgfbr1_symbol))
tgfbr1_len = len(lds1_phenopackets)
print(f"{len(lds1_phenopackets)} LDS1 phenopackets")

41 LDS1 phenopackets


# LDS2
[Loeys-Dietz syndrome 2 (OMIM:610168)](https://omim.org/entry/610168) is caused by heterozygous mutation in the TGFBR2 gene.

In [3]:
tgfbr2_symbol = 'TGFBR2'
tgfbr2_mane_tx_id = 'NM_003242.6'
tgfbr2_mane_protein_id = 'NP_003233.4' # TGF-beta receptor type-2 isoform B precursor
lds2_disease_id = "OMIM:610168"

from ppktstore.registry import configure_phenopacket_registry
phenopacket_registry = configure_phenopacket_registry()
with phenopacket_registry.open_phenopacket_store("0.1.24") as ps:
    lds2_phenopackets = tuple(ps.iter_cohort_phenopackets(tgfbr2_symbol))
print(f"{len(lds2_phenopackets)} LDS2 phenopackets")

53 LDS2 phenopackets


In [4]:
from gpsea.preprocessing import configure_caching_cohort_creator, load_phenopackets

lds_phenopackets = list()
lds_phenopackets.extend(lds1_phenopackets)
lds_phenopackets.extend(lds2_phenopackets)

cohort_creator = configure_caching_cohort_creator(hpo)
cohort, validation = load_phenopackets(
    phenopackets=lds_phenopackets, 
    cohort_creator=cohort_creator,
)

validation.summarize()

Individuals Processed: 100%|██████████| 94/94 [00:01<00:00, 71.42 individuals/s] 
Validated under permissive policy
No errors or warnings were found


In [5]:
from gpsea.analysis.pcats import configure_hpo_term_analysis
from gpsea.analysis.clf import prepare_classifiers_for_terms_of_interest

analysis = configure_hpo_term_analysis(hpo)

pheno_clfs = prepare_classifiers_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
)

In [6]:
from gpsea.analysis.clf import diagnosis_classifier
from gpsea.view import MtcStatsViewer


lds_1_2_disease_clf = diagnosis_classifier(
    diagnoses=(lds1_disease_id, lds2_disease_id),
    labels=('LDS1', 'LDS2'),
)
lds_1_2_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_clf=lds_1_2_disease_clf,
    pheno_clfs=pheno_clfs,
)

viewer = MtcStatsViewer()
viewer.process(lds_1_2_result)

Reason,Count
Skip terms if all counts are identical to counts for a child term,3
"Skipping ""general"" level terms",73
Skipping terms that are rare on the cohort level (in less than 40% of the cohort members),254


In [7]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=lds_1_2_result)

Diagnosis,OMIM:609192,OMIM:610168,Corrected p values,p values
Epicanthus [HP:0000286],3/13 (23%),19/30 (63%),0.674224,0.021695
Talipes equinovarus [HP:0001762],2/17 (12%),11/24 (46%),0.674224,0.039114
Bifid uvula [HP:0000193],4/16 (25%),20/36 (56%),0.674224,0.069543
Scoliosis [HP:0002650],18/21 (86%),23/38 (61%),0.674224,0.074914
Dermal translucency [HP:0010648],11/16 (69%),9/22 (41%),0.686332,0.111858
Arterial tortuosity [HP:0005116],9/17 (53%),23/30 (77%),0.686332,0.114389
Pectus excavatum [HP:0000767],3/14 (21%),12/25 (48%),0.821553,0.170555
Pectus carinatum [HP:0000768],7/18 (39%),5/26 (19%),0.821553,0.182567
Mitral regurgitation [HP:0001653],9/22 (41%),4/17 (24%),1.000000,0.317881
Joint hypermobility [HP:0001382],12/19 (63%),25/32 (78%),1.000000,0.333504


# Summary

In [8]:
from gpseacs.report import GpseaAnalysisReport, GPAnalysisResultSummary

f_results = (
  GPAnalysisResultSummary.from_multi( result=lds_1_2_result  ),
)


caption = """."""
report = GpseaAnalysisReport(name="LDS 1 and 2", 
                             cohort=cohort, 
                             fet_results=f_results,
                             gene_symbol="n/a",
                             mane_tx_id="n/a",
                             mane_protein_id="n/a",
                             caption=caption)

In [9]:
from gpseacs.report import GpseaNotebookSummarizer
summarizer = GpseaNotebookSummarizer(hpo=hpo, gpsea_version=gpsea.__version__)
summarizer.summarize_report(report=report)

Genotype (A),Genotype (B),Tests performed,Significant tests
OMIM:609192,OMIM:610168,36,0


In [10]:
summarizer.process_latex(report=report)

Output to ../../supplement/tex/LDS_1_and_2_summary_draft.tex
